In [1]:
import tweepy
import json
import datetime
import pandas as pd

In [2]:
from tqdm import tqdm

In [19]:
Bearertoken = "PUT YOUR BEARERTOKEN HERE"

In [20]:
# define the api to get users timeline tweets

auth = tweepy.OAuth2BearerHandler(Bearertoken)
api = tweepy.API(auth, wait_on_rate_limit=True)
tweets_getter = tweepy.Client(bearer_token=Bearertoken, wait_on_rate_limit=True)

In [22]:
# define the time window be to in year of 2020
start_time = datetime.datetime.strptime("2020-01-01", '%Y-%m-%d')
end_time = datetime.datetime.strptime("2020-12-31", '%Y-%m-%d')

# define the fields of data we will collect
Expansions='author_id'
Tweet_fields=['created_at', 'geo', 'author_id', 'in_reply_to_user_id', 'lang']
User_fields=['id', 'username', 'created_at', 'public_metrics']

In [23]:
# define function to write the tweets data into a pandas dataframe
def get_user_tweets_df(user_id):
    import time
    # define the query for the request to the api
    Query = 'lang:en from:'+str(user_id)
    tweets_list = []
    # define the request into a paginator
    for response in tweepy.Paginator(tweets_getter.search_all_tweets, query=Query, end_time=end_time,
                                expansions=Expansions,
                                tweet_fields=Tweet_fields,
                                start_time=start_time,               
                                max_results=500):
        response_data = response.data
        if isinstance(response_data,list):
            for tweet in response_data:
                tweet_data = pd.DataFrame({'tweet_id':tweet.id,
                                           'text':tweet.text,
                                           'created_at':tweet.created_at,
                                           'geo':tweet.geo,
                                           'author_id':tweet.author_id,
                                           'in_reply_to_user_id':tweet.in_reply_to_user_id,
                                           'lang':tweet.lang}, index=[0])
                tweets_list.append(tweet_data)
        else:
            pass
        time.sleep(1)
        
    # concatenate returned tweets into a pandas dataframe
    if len(tweets_list) > 0:
        user_tweets_df = pd.concat(tweets_list, ignore_index=True)
    else:
        user_tweets_df = pd.DataFrame()
        
    return(user_tweets_df)

In [52]:
# load the list of filtered users
with open('/data/user_tweets/filtered_user_list.txt', 'r') as f:
    user_list = [int(x) for x in f.read().split()]

In [ ]:
for i in tqdm(range(len(user_list))):
    # start downloading the tweets data and store into a pandas dataframe
    user_tweets_df = get_user_tweets_df(user_list[i])
    
    save_file_name = '/data/user_tweets/'+str(user_list[i])+'.csv'
     # save the dataframe into a csv file
    user_tweets_df.to_csv(save_file_name)